# Pescador demo

This notebook illustrates some of the basic functionality of [pescador](https://github.com/bmcfee/pescador): a package to facilitate iterative learning from data streams (implemented as python generators).

In [1]:
import pescador

import numpy as np
np.set_printoptions(precision=4)
import sklearn
import sklearn.datasets
import sklearn.linear_model
import sklearn.cross_validation
import sklearn.metrics

In [2]:
def data_generator(X, Y, m=20, scale = 1e-1):
    '''A gaussian noise generator for data
    
    Parameters
    ----------
    X : ndarray
        features, n_samples by dimensions
        
    Y : ndarray
        labels, n_samples
        
    m : int
        size of the minibatches to generate
        
    scale : float > 0
        scale of the noise to add
        
    Generates
    ---------
    batch
        An infinite stream of batch dictionaries
        batch = dict(X=X[i], Y=Y[i])
    '''
    
    X = np.atleast_2d(X)
    Y = np.atleast_1d(Y)

    
    n, d = X.shape
    
    while True:
        i = np.random.randint(0, n, size=m)
        
        noise = scale * np.random.randn(m, d)
        
        yield {'X': X[i] + noise, 'Y': Y[i]}

In [3]:
# Load up the iris dataset for the demo
data = sklearn.datasets.load_iris()
X, Y = data.data, data.target
classes = np.unique(Y)

In [4]:
# What does the data stream look like?

# First, we'll wrap the generator function in a Streamer object.
# This is necessary for a few reasons, notably so that we can re-instantiate
# the generator multiple times (eg once per epoch)

stream = pescador.Streamer(data_generator, X, Y)

# The buffer_batch() function takes a batch stream as input, and
# carves it into batches of up to buffer_size (3, in this case) samples
# the buffer size can be larger or smaller than the native size of the input batches
for q in pescador.buffer_batch(stream.generate(max_batches=1), 3):
    print q

{'Y': array([1, 1, 1]), 'X': array([[ 5.6131,  2.7952,  4.5718,  1.2556],
       [ 6.3293,  3.319 ,  4.6284,  1.783 ],
       [ 5.0094,  1.9999,  3.3762,  1.0161]])}
{'Y': array([0, 0, 0]), 'X': array([[ 4.6549,  3.0582,  1.2186,  0.1576],
       [ 5.1507,  3.1515,  1.5928,  0.1946],
       [ 4.5464,  3.2414,  1.2479,  0.0669]])}
{'Y': array([0, 2, 1]), 'X': array([[ 5.4137,  3.4423,  1.8064,  0.2326],
       [ 6.5472,  3.0455,  5.2515,  2.4077],
       [ 5.8149,  2.7948,  4.1071,  0.9175]])}
{'Y': array([0, 1, 1]), 'X': array([[ 4.4945,  2.8507,  1.4083,  0.0522],
       [ 5.9375,  2.6966,  4.5145,  1.463 ],
       [ 6.3355,  3.0281,  4.3717,  1.4244]])}
{'Y': array([0, 1, 1]), 'X': array([[ 4.7427,  2.9923,  1.4225,  0.1629],
       [ 6.1609,  2.3934,  4.3818,  1.3451],
       [ 5.3197,  2.4496,  3.8024,  1.1289]])}
{'Y': array([0, 2, 0]), 'X': array([[ 4.9996,  3.7724,  1.3805,  0.3327],
       [ 6.6495,  3.0937,  5.3445,  2.1362],
       [ 4.8606,  3.0358,  1.2964,  0.2183]])}
{'Y'

# Benchmarking
We can benchmark our learner's efficiency by running a couple of experiments on the Iris dataset.

Our classifier will be L1-regularized logistic regression.

In [9]:
%%time
for train, test in sklearn.cross_validation.ShuffleSplit(len(X),
                                                         n_iter=2,
                                                         test_size=0.2):
    
    # Make an SGD learner, nothing fancy here
    classifier = sklearn.linear_model.SGDClassifier(verbose=0, 
                                                    loss='log',
                                                    penalty='l1', 
                                                    n_iter=1)
    
    # Make a streamable wrapper
    model = pescador.StreamLearner(classifier)
    
    # Again, build a streamer object
    stream = pescador.Streamer(data_generator, X[train], Y[train])
    
    samples = stream.generate(max_batches=5e3//20)
    
    # And train the model on the stream.
    # iter_fit() works just like partial_fit(), except that the input is a generator.
    model.iter_fit(samples, classes=classes)
    
    # How's it do on the test set?
    print 'Test-set accuracy: {:.3f}'.format(sklearn.metrics.accuracy_score(Y[test], model.predict(X[test])))
    print '# Steps: ' + str(model.estimator.t_)

Test-set accuracy: 0.833
# Steps: 5001.0
Test-set accuracy: 1.000
# Steps: 5001.0
CPU times: user 190 ms, sys: 3.32 ms, total: 194 ms
Wall time: 192 ms


# Parallelism

It's possible that the learner is more or less efficient than the data generator.  If the data generator has higher latency than the learner (SGDClassifier), then this will slow down the learning.

Pescador uses zeromq to parallelize data stream generation, effectively decoupling it from the learner.

In [8]:
%%time
for train, test in sklearn.cross_validation.ShuffleSplit(len(X), n_iter=2, test_size=0.2):
    
    # Make an SGD learner, nothing fancy here
    classifier = sklearn.linear_model.SGDClassifier(verbose=0, 
                                                    loss='log',
                                                    penalty='l1', 
                                                    n_iter=1)
    
    # Make a streamable wrapper
    model = pescador.StreamLearner(classifier)
    
    # First, turn the data_generator function into a Streamer object
    stream = pescador.Streamer(data_generator, X[train], Y[train])
    
    # Then, send this thread to a second process
    zmq_stream = pescador.zmq_stream(5156, stream, max_batches=5e3)
    
    # Run the output through a second buffer for mini-batch training
    #samples = pescador.buffer_batch(zmq_stream, 20)
    samples = zmq_stream
    
    # And fit on the stream
    model.iter_fit(samples, classes=classes)
    
    # How's it do on the test set?
    print 'Test-set accuracy: {:.3f}'.format(sklearn.metrics.accuracy_score(Y[test], model.predict(X[test])))
    print '# Steps: ' + str(model.estimator.t_)

Test-set accuracy: 0.967
# Steps: 4841.0
Test-set accuracy: 0.900
# Steps: 5621.0
CPU times: user 359 ms, sys: 29.5 ms, total: 388 ms
Wall time: 335 ms
